In [1]:
import torch
import torch.optim as optim
import numpy as np
from transformer import Transformer
from metrics import Evaluator
from datasets import load_dataset
from utils import *
from training import *
from tokenizer import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

/Users/tonimo/Documents/Developing/Repos/Deep-Learning/deep-learning-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cpu


In [2]:
traindict = load_dataset("opus100", "de-en", split="train")
testdict = load_dataset("opus100", "de-en", split="test")

Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)
Found cached dataset opus100 (/Users/tonimo/.cache/huggingface/datasets/opus100/de-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)


In [3]:
train_inputs, train_labels = get_split(traindict, "en", "de", size=100)
test_inputs, test_labels = get_split(testdict, "en", "de", size=10)
trainset = Dataset(train_inputs, train_labels)
testset = Dataset(test_inputs, test_labels)

In [ ]:
trainframe = trainset.dataframe()
trainframe.head()

,inputs,labels
0,It's greed that it's gonna be the death of you...,Deine Habgier wird noch dein Tod sein.
1,Vega.,- Vega.
2,Just say when.,Sagen Sie einfach stopp.
3,- Wait.,- Warte.
4,I don't wanna be here.,Ich will nicht hier sein.


In [ ]:
testframe = testset.dataframe()
testframe.head()

,inputs,labels
0,"By clicking on 'Save profile', you the user ag...",Die Nutzungsbedingungen werden durch das Klick...
1,I wanted to show you something first.,Ich wollte dir erst noch etwas zeigen.
2,You have suffered because of Shinkichi.,Du musstest wegen Shinkichi leiden.
3,"moodle:bg-bab: Calendar: Day view: Friday, 25 ...",moodle:bg-bab: Kalender: Tagesansicht: Freitag...
4,"I mean, most people, they see another person w...","Ich meine, die meisten Leuten sehen eine ander..."


In [ ]:
print(trainframe.isnull().values.any())
trainframe.describe()

False


,inputs,labels
count,100,100
unique,100,100
top,It's greed that it's gonna be the death of you...,Deine Habgier wird noch dein Tod sein.
freq,1,1


In [ ]:
print(testframe.isnull().values.any())
testframe.describe()

False


,inputs,labels
count,10,10
unique,10,10
top,"By clicking on 'Save profile', you the user ag...",Die Nutzungsbedingungen werden durch das Klick...
freq,1,1


In [ ]:
trainset.sample()

[("We're leaving.", 'Wir fahren.')]

In [ ]:
testset.sample()

[('I wanted to show you something first.',
  'Ich wollte dir erst noch etwas zeigen.')]

In [ ]:
corpus_en = trainset.corpus(split=0) + testset.corpus(split=0)
corpus_de = trainset.corpus(split=1) + testset.corpus(split=1)
tokenizer_en = Nerdimizer()
tokenizer_de = Nerdimizer()
tokenizer_en.train(corpus_en, size=50000)
tokenizer_de.train(corpus_de, size=50000)
translator = Translator(tokenizer_en, tokenizer_de)

In [ ]:
en_vocab, de_vocab = translator.vocab_size()
maxlen_train = trainset.maxlen(translator)
maxlen_test = testset.maxlen(translator)
maxlen = min(maxlen_train, maxlen_test, 200)
start, end, pad = tokenizer_en["[S]"], tokenizer_en["[E]"], tokenizer_en["[P]"]
print(f"Number of input tokens: {len(tokenizer_en)}\nNumber of output tokens: {len(tokenizer_de)}")

Number of input tokens: 860
Number of output tokens: 1064


In [ ]:
translator.padon(maxlen, end=True, pad_id=pad)
translator.truncon(maxlen, end=True)
print(f"Maxlen: {maxlen}")

Maxlen: 46


In [ ]:
tokenized_trainset = trainset.tokenized(translator, model=True)
dataloader = tokenized_trainset.dataloader(batch_size=32, drop_last=False)

In [ ]:
model = Transformer(en_vocab, de_vocab, maxlen, pad_id=pad, dm=256, nhead=8, layers=3, dff=1024)
optimizer = optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=10e-9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=10)
evaluator = Evaluator(testset, translator, start, end, maxlen, sample=10, ngrams=4, threshold=20, 
                    mode="geometric", device=device)
clock = Clock()
checkpoint = Checkpoint(model, optimizer, scheduler, evaluator, clock, epochs=3, 
                    path="checkpoints/english-german", overwrite=False)
model.to(device);

In [ ]:
train(dataloader, model, optimizer, scheduler, evaluator, checkpoint, clock,
    epochs=5, warmups=100, verbose=True, device=device)

Training Started
----------------------------------------------------------------------
Epoch 1 Complete | Epoch Duration: 00:00:08 | Elapsed Time: 00:00:08 |
Metrics | Epoch Loss: 7.0040 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
----------------------------------------------------------------------
Epoch 2 Complete | Epoch Duration: 00:00:03 | Elapsed Time: 00:00:10 |
Metrics | Epoch Loss: 6.8147 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
----------------------------------------------------------------------
Epoch 3 Complete | Epoch Duration: 00:00:02 | Elapsed Time: 00:00:13 |
Metrics | Epoch Loss: 6.6449 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: True |
----------------------------------------------------------------------
Epoch 4 Complete | Epoch Duration: 00:00:02 | Elapsed Time: 00:00:15 |
Metrics | Epoch Loss: 6.5125 | BLEU Score: 0.0 | 
Other Inf

6.665164875984192

In [ ]:
checkpoint = Checkpoint(model, optimizer, scheduler)
checkpoint.load_checkpoint(path="checkpoints/english-german-3")

{'model_params': OrderedDict([('linear', tensor([[-0.0613,  0.0100,  0.0456,  ..., -0.0484, -0.0446,  0.0152],
        [-0.0619,  0.0342, -0.0665,  ...,  0.0160,  0.0370, -0.0554],
        [ 0.0070, -0.0553, -0.0040,  ..., -0.0354,  0.0557, -0.0557],
        ...,
        [-0.0227, -0.0229, -0.0167,  ..., -0.0311, -0.0668,  0.0120],
        [-0.0053, -0.0327, -0.0093,  ...,  0.0281,  0.0142, -0.0495],
        [ 0.0539, -0.0481, -0.0057,  ..., -0.0336,  0.0165, -0.0328]])), ('encoder.stack.0.multihead.wq.weight', tensor([[-0.0975, -0.0643, -0.0005,  ..., -0.0722,  0.0629,  0.0989],
        [ 0.0944,  0.0076, -0.0445,  ..., -0.0472, -0.0322,  0.0129],
        [-0.0859,  0.0202, -0.0375,  ..., -0.0720, -0.0293,  0.0797],
        ...,
        [-0.0255,  0.0225, -0.0358,  ...,  0.0027, -0.0018,  0.0186],
        [-0.0891, -0.0958,  0.0194,  ...,  0.0039, -0.0067,  0.0047],
        [ 0.0634,  0.0772,  0.0012,  ...,  0.1001,  0.1043,  0.0895]])), ('encoder.stack.0.multihead.wk.weight', tensor(

In [ ]:
print(checkpoint.clock)

None


In [ ]:
retrain(dataloader, checkpoint, 11)

Training Resumed
----------------------------------------------------------------------
Epoch 4 Complete | 
Metrics | Epoch Loss: 6.5073 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
----------------------------------------------------------------------
Epoch 5 Complete | 
Metrics | Epoch Loss: 6.3664 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
----------------------------------------------------------------------
Epoch 6 Complete | 
Metrics | Epoch Loss: 6.2524 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: True |
----------------------------------------------------------------------
Epoch 7 Complete | 
Metrics | Epoch Loss: 6.1620 | BLEU Score: 0.0 | 
Other Info | Scheduler Warmup Step: True | Checkpoint Saved: False |
----------------------------------------------------------------------
Epoch 8 Complete | 
Metrics | Epoch Loss: 6.1131 | BLEU Score: 0.0 | 
Oth

KeyboardInterrupt: 